In [ ]:
# First off we're going to get some basic understanding.
How many rows and columns
What they mean
What our target is
Summary statistics
What we can say of the domain

In [ ]:
Here's the source I'm looking at:
http://www.amstat.org/publications/jse/v19n3/decock/AmesHousing.xls. 
    
Here's the documentation available on the data:
https://ww2.amstat.org/publications/jse/v19n3/decock/DataDocumentation.txt

In [2]:
#Getting in some generally useful libraries

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as st
from IPython.display import display
%matplotlib inline

In [15]:
#Acquire the Data
#I did a curl to my local, so I can pull data from there.
# curl -O http://www.amstat.org/publications/jse/v19n3/decock/AmesHousing.xls
# For some reason doing a head after a curl looked unformatted, but pandas reads it right.
#The alternate is to directly pull from the URL.

hous_df = pd.read_excel('Data/AmesHousing.xls', delimiter=r"\s+")
hous_df.shape

#That tallies

(2930, 82)

In [ ]:
#What the data means :

Data is from 2010

23 nominal    
23 ordinal  
14 discrete 
20 continuous variables              
2 additional observation identifiers 
    
That's confusing. Cuz we know continuous and discrete. 

Looking at the dataset it looks like

Discrete and continuos is used for numerical values. Ex Discrete - Year ; Continuous - Lot Size
Nominal and ordinal are used for labels. Ex. Nominal - type of road ; Ordinal - Quality

For our purpose,  we could use all nominals and discretes as categorical.
However ordinals have an order, so it's tricky, but we are taking it as categorical because :
https://www.ma.utexas.edu/users/mks/statmistakes/ordinal.html
    
All continuos can be taken as numerics.

In [45]:
#Let's look at how the data looks. The headers are already there. Awesome
hous_df.head()
hous_df.iloc[:5,15:]
#There are too many variables so instead of reading it all, we'll understand it later through charts
#Our target is the housing price

,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,BrkFace,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,Norm,1Fam,1Story,7,5,1968,1968,Hip,CompShg,BrkFace,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [42]:
#Some Summary Statistics
# Oh before that, we got to remove the non-numerics, cuz why would we need the mean of labels, right?

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

#Used copy() here because pass-by-refernce is confusing
hous_n = hous_df.copy().select_dtypes(include=numerics)
hous_n.shape
list(hous_n)

['Order',
 'PID',
 'MS SubClass',
 'Lot Frontage',
 'Lot Area',
 'Overall Qual',
 'Overall Cond',
 'Year Built',
 'Year Remod/Add',
 'Mas Vnr Area',
 'BsmtFin SF 1',
 'BsmtFin SF 2',
 'Bsmt Unf SF',
 'Total Bsmt SF',
 '1st Flr SF',
 '2nd Flr SF',
 'Low Qual Fin SF',
 'Gr Liv Area',
 'Bsmt Full Bath',
 'Bsmt Half Bath',
 'Full Bath',
 'Half Bath',
 'Bedroom AbvGr',
 'Kitchen AbvGr',
 'TotRms AbvGrd',
 'Fireplaces',
 'Garage Yr Blt',
 'Garage Cars',
 'Garage Area',
 'Wood Deck SF',
 'Open Porch SF',
 'Enclosed Porch',
 '3Ssn Porch',
 'Screen Porch',
 'Pool Area',
 'Misc Val',
 'Mo Sold',
 'Yr Sold',
 'SalePrice']

In [48]:
# We can later convert the non-numerics into dummies / one-hot encode it.
# https://stackoverflow.com/questions/29528628/how-to-specify-a-variable-in-pandas-as-ordinal-categorical
# What this says is use factorize to convert discrete to categoricals.
# But use the custom mapper to convert ordinal to categorical, preserving the order
# So basically everything gets converted to numeric, but we factorize discrete, and ordinal, we makes dummies from nominal

# Factorizing the discretes
# Order, Year Built, Year Remod/Add , Bsmt Full Bath , Bsmt Half Bath ,Full Bath ,Half Bath ,Bedroom,Kitchen 
# TotRmsAbvGrd, Fireplaces, Garage Yr Blt, Garage Cars, Mo Sold, Yr Sold

hous_n['Order'] = hous_n['Order'].astype('category')
hous_n['Year Built'] = hous_n['Year Built'].astype('category')
hous_n['Year Remod/Add'] = hous_n['Year Remod/Add'].astype('category')
hous_n['Bsmt Full Bath'] = hous_n['Bsmt Full Bath'].astype('category')
hous_n['Bsmt Half Bath'] = hous_n['Bsmt Half Bath'].astype('category')
hous_n['Full Bath'] = hous_n['Full Bath'].astype('category')
hous_n['Half Bath'] = hous_n['Half Bath'].astype('category')
hous_n['Bedroom AbvGr'] = hous_n['Bedroom AbvGr'].astype('category')
hous_n['Kitchen AbvGr'] = hous_n['Kitchen AbvGr'].astype('category')
hous_n['TotRms AbvGrd'] = hous_n['TotRms AbvGrd'].astype('category')
hous_n['Fireplaces'] = hous_n['Fireplaces'].astype('category')
hous_n['Garage Yr Blt'] = hous_n['Garage Yr Blt'].astype('category')
hous_n['Garage Cars'] = hous_n['Garage Cars'].astype('category')
hous_n['Mo Sold'] = hous_n['Mo Sold'].astype('category')
hous_n['Yr Sold'] = hous_n['Yr Sold'].astype('category')

In [51]:
hous_nn = hous_n.copy().select_dtypes(include=numerics)
hous_nn.shape

(2930, 39)

In [53]:
#Now finally some summary statistics
hous_stats = hous_nn.describe().T

#Adding skew and kurtosis
from scipy.stats import skew
from scipy.stats import kurtosis
hous_stats['skew'] = skew(hous_nn)
hous_stats['kurtosis'] = kurtosis(hous_nn)

hous_stats

,count,mean,std,min,25%,50%,75%,max,skew,kurtosis
PID,2930.0,7.144645e+08,1.887308e+08,526301100.0,5.284770e+08,535453620.0,9.071811e+08,1.007100e+09,0.055857,-1.993790
MS SubClass,2930.0,5.738737e+01,4.263802e+01,20.0,2.000000e+01,50.0,7.000000e+01,1.900000e+02,1.356884,1.382363
Lot Frontage,2440.0,6.922459e+01,2.336533e+01,21.0,5.800000e+01,68.0,8.000000e+01,3.130000e+02,NaN,NaN
Lot Area,2930.0,1.014792e+04,7.880018e+03,1300.0,7.440250e+03,9436.5,1.155525e+04,2.152450e+05,12.814334,264.569581
Overall Qual,2930.0,6.094881e+00,1.411026e+00,1.0,5.000000e+00,6.0,7.000000e+00,1.000000e+01,0.190536,0.050276
Overall Cond,2930.0,5.563140e+00,1.111537e+00,1.0,5.000000e+00,5.0,6.000000e+00,9.000000e+00,0.574135,1.486859
Mas Vnr Area,2907.0,1.018968e+02,1.791126e+02,0.0,0.000000e+00,0.0,1.640000e+02,1.600000e+03,NaN,NaN
BsmtFin SF 1,2929.0,4.426296e+02,4.555908e+02,0.0,0.000000e+00,370.0,7.340000e+02,5.644000e+03,NaN,NaN
BsmtFin SF 2,2929.0,4.972243e+01,1.691685e+02,0.0,0.000000e+00,0.0,0.000000e+00,1.526000e+03,NaN,NaN
Bsmt Unf SF,2929.0,5.592625e+02,4.394942e+02,0.0,2.190000e+02,466.0,8.020000e+02,2.336000e+03,NaN,NaN


In [55]:
#Why would skew be NaN?
# I could ignore NaNs for skew, but what's the impact?

hous_stats['skew1'] = skew(hous_nn.dropna())
hous_stats['kurtosis1'] = kurtosis(hous_nn.dropna())
hous_stats

,count,mean,std,min,25%,50%,75%,max,skew,kurtosis,skew1,kurtosis1
PID,2930.0,7.144645e+08,1.887308e+08,526301100.0,5.284770e+08,535453620.0,9.071811e+08,1.007100e+09,0.055857,-1.993790,0.028200,-1.996654
MS SubClass,2930.0,5.738737e+01,4.263802e+01,20.0,2.000000e+01,50.0,7.000000e+01,1.900000e+02,1.356884,1.382363,1.367406,1.310615
Lot Frontage,2440.0,6.922459e+01,2.336533e+01,21.0,5.800000e+01,68.0,8.000000e+01,3.130000e+02,NaN,NaN,1.512203,11.320855
Lot Area,2930.0,1.014792e+04,7.880018e+03,1300.0,7.440250e+03,9436.5,1.155525e+04,2.152450e+05,12.814334,264.569581,15.047647,436.213683
Overall Qual,2930.0,6.094881e+00,1.411026e+00,1.0,5.000000e+00,6.0,7.000000e+00,1.000000e+01,0.190536,0.050276,0.218504,-0.028461
Overall Cond,2930.0,5.563140e+00,1.111537e+00,1.0,5.000000e+00,5.0,6.000000e+00,9.000000e+00,0.574135,1.486859,0.616352,1.285430
Mas Vnr Area,2907.0,1.018968e+02,1.791126e+02,0.0,0.000000e+00,0.0,1.640000e+02,1.600000e+03,NaN,NaN,2.681472,9.821324
BsmtFin SF 1,2929.0,4.426296e+02,4.555908e+02,0.0,0.000000e+00,370.0,7.340000e+02,5.644000e+03,NaN,NaN,1.607869,8.055880
BsmtFin SF 2,2929.0,4.972243e+01,1.691685e+02,0.0,0.000000e+00,0.0,0.000000e+00,1.526000e+03,NaN,NaN,4.183208,19.162859
Bsmt Unf SF,2929.0,5.592625e+02,4.394942e+02,0.0,2.190000e+02,466.0,8.020000e+02,2.336000e+03,NaN,NaN,0.873202,0.298874


In [ ]:
So it is skewed, and we may want to deskew it 
In the next section , EDA, we will look at the categorical variables
Look at the normality
And normalize data as needed

In [ ]:
# Complicated stuff about pandas.factorize
# https://www.programcreek.com/python/example/101364/pandas.factorize

In [ ]:
## The type of data science or machine learning you intend to do

Since target is continuous, regression seems like a good solution

# A benchmark model
Could be mean

# A performance metric used to assess the performance of your proposed solution
MSE(outliers will impact)
R^2(1-rss/tss - explained variance)
Rss = mse/
RMSE
MAE(less skewed will impact)